In [31]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

In [32]:
path = "Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_282_286/"

### input data

In [18]:
sample_50K = pd.read_adls(path + "Input/air_282_regression_50k_sample.csv")
print(sample_50K.shape)
# pd.DataFrame(sample_50K.isnull().sum())

(50000, 34)


/anaconda/envs/data-science/lib/python3.10/site-packages/dsgtools/azure/adls.py:239: DtypeWarning: Columns (6,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data = reader(file, **kwargs)


In [36]:
cases = pd.read_adls(path + "Input/air_282_weight_below_threshold_sample.csv", dtype = str)
print(cases.shape)
# pd.DataFrame(cases.isnull().sum())

(562, 16)


In [37]:
cases.columns

Index(['accountnumber', 'companyname', 'alternatecompanyname', 'addr', 'city',
       'state', 'zip', 'businessphone', 'taxidnumber', 'historydate', 'powid',
       'proxid', 'seleid', 'orgid', 'ultid', 'overallweight'],
      dtype='object')

In [41]:
test.historydateyyyymm.unique()

array([202203, 202204, 202206, 202201, 202108, 202302, 202301, 202111,
       202107, 202202, 202205, 202109, 202112, 202110])

In [28]:
temp = baseline.loc[(baseline["seleid"].astype(float) == 0) & (baseline.b2bcnt2y >= 0), "accountnumber"].copy()
cases = cases[cases.accountnumber.isin(temp.to_list())]
print(cases.shape)

(97, 16)


In [30]:
cases.to_csv("./_temp/sample_QA_282_1016.csv", index = False)

In [23]:
baseline.shape

(50560, 2308)

In [ ]:
temp = baseline[(baseline["seleid"].astype(float) == 0) & (baseline.b2bcnt2y >= 0), "accountnumber"].copy()

(103, 2308)

In [35]:
temp = test[(test["seleid"].astype(float) == 0) & (test.b2bcnt2y >= 0)].copy()
temp.shape

(0, 2304)

### BusShell31:

In [33]:
from dsgtools import azure
shell = pd.read_adls(path + "data/air_282_business_shell_v31_results_w20230614-123947_sas_layout_busshell.csv", encoding='iso-8859-1')
baseline = pd.read_adls(path + "data/air_282_business_shell_v31_results_w20230615-074319.csv", encoding='iso-8859-1')
test = pd.read_adls(path + "data/air_282_business_shell_v31_results_w20230615-091509.csv", encoding='iso-8859-1')

print(shell.shape)
print(baseline.shape)
print(test.shape)

/anaconda/envs/data-science/lib/python3.10/site-packages/dsgtools/azure/adls.py:239: DtypeWarning: Columns (22,23,39,51,52,69,71,360,682,3098) have mixed types. Specify dtype option on import or set low_memory=False.
  data = reader(file, **kwargs)
/anaconda/envs/data-science/lib/python3.10/site-packages/dsgtools/azure/adls.py:239: DtypeWarning: Columns (130,2302) have mixed types. Specify dtype option on import or set low_memory=False.
  data = reader(file, **kwargs)
/anaconda/envs/data-science/lib/python3.10/site-packages/dsgtools/azure/adls.py:239: DtypeWarning: Columns (130,2302) have mixed types. Specify dtype option on import or set low_memory=False.
  data = reader(file, **kwargs)


(50560, 3115)
(50562, 2304)
(50561, 2304)


In [5]:
fmt_1 = make_format(cuts = [-np.inf, 0, np.inf])
fmt_2 = make_format(cuts = [-np.inf, 0, 43, np.inf])
freq(shell.id_seleid.astype(float), shell.id_weight.astype(float), format = [fmt_1, fmt_2], cross=False, observed = True)

Count       Pct  Cuml Count  Cuml Pct
id_seleid id_weight                                         
<= 0      <= 0        8481.0  0.167741      8481.0  0.167741
1+        1-43         207.0  0.004094      8688.0  0.171835
          44+        41867.0  0.828066     50555.0  0.999901
Missing   Missing        5.0  0.000099     50560.0  1.000000

In [6]:
shell["change_flag"] = np.where((shell.id_seleid > 0) & (shell.id_weight < 44), 1, 0)
freq(shell.change_flag)

,Count,Pct,Cuml Count,Cuml Pct
change_flag,,,,
0,50353.0,0.995906,50353.0,0.995906
1,207.0,0.004094,50560.0,1.000000


In [7]:
shell = shell[["account", "id_seleid", "id_weight", "change_flag"]]
baseline = baseline.merge(shell, left_on = "accountnumber", right_on = 'account')
test = test.merge(shell, left_on = "accountnumber", right_on = 'account')

In [ ]:
print(baseline.shape)
print(test.shape)

(50560, 2308)
(50559, 2308)


In [ ]:
baseline["seleid_same"] = np.where(baseline.seleid == baseline.id_seleid, 1, 0)
freq(baseline["seleid_same"], baseline.change_flag, cross = False, observed = True)

Count       Pct  Cuml Count  Cuml Pct
seleid_same change_flag                                         
1           0            50345.0  0.995748     50345.0  0.995748
0           0                8.0  0.000158     50353.0  0.995906
            1              207.0  0.004094     50560.0  1.000000

In [ ]:
test["seleid_same"] = np.where(test.seleid == test.id_seleid, 1, 0)
freq(test["seleid_same"], test.change_flag, cross = False, observed = True)

Count       Pct  Cuml Count  Cuml Pct
seleid_same change_flag                                         
1           0            50344.0  0.995748     50344.0  0.995748
            1              207.0  0.004094     50551.0  0.999842
0           0                8.0  0.000158     50559.0  1.000000

In [ ]:
import datacompy

In [ ]:
# base_no_impact = baseline[(baseline["seleid_same"] == 1) & (baseline["change_flag"] == 0)].copy()
# test_no_impact = test[(test["seleid_same"] == 1) & (test["change_flag"] == 0)].copy()

In [ ]:
# base_no_impact = baseline[(baseline["seleid_same"] == 1) & (baseline["change_flag"] == 0)].copy()
# test_no_impact = test[(test["seleid_same"] == 1) & (test["change_flag"] == 0)].copy()

# scout_compare = datacompy.Compare(
#             base_no_impact,
#             test_no_impact,
#             join_columns='accountnumber', #You can also specify a list of columns
#             abs_tol=0.0001,
#             rel_tol=0,
#             df1_name='base',
#             df2_name='test')

wb = TableWriter(filename = "./_temp/scout_no_impact.xlsx", options={'nan_inf_to_errors': True})
wb.write_table(
    pd.DataFrame(scout_compare.column_stats).fillna("n/a"),
    sheetname="scout_no_impact",
)
wb.close() 

In [ ]:
# print(scout_compare.report())
from dsgtools.reporting import col_summary

wb = TableWriter(filename = "./_temp/change_baseline.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
wb.write_table(
    col_summary(baseline[(baseline["change_flag"] == 1)].copy(), exceptions = [-1, -97, -98, -99]).T.fillna(""),
    sheetname="change_baseline",
)
wb.close() 

In [ ]:
test[(test["change_flag"] == 1)].columns

Index(['accountnumber', 'historydateyyyymm', 'bus_company_name', 'powid',
       'proxid', 'seleid', 'orgid', 'ultid', 'lnlexidsele',
       'inputcheckbusname',
       ...
       'model10rc4', 'model10rc5', 'model10rc6', 'errorcode', 'time_ms',
       'account', 'id_seleid', 'id_weight', 'change_flag', 'seleid_same'],
      dtype='object', length=2309)

### BusShell21:

In [5]:
shell = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_282_286/data/0628_business_shell_v21_results_w20230626-104524_sas_layout_busshell.csv", encoding='iso-8859-1')
baseline = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_282_286/data/0627_sba_v1_sbfe_busshell_w20230626-111450", encoding='iso-8859-1', reader = pd.read_csv)
test = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_282_286/data/0627_sba_v1_sbfe_busshell_w20230626-120006", encoding='iso-8859-1', reader = pd.read_csv)

print(shell.shape)
print(baseline.shape)
print(test.shape)

/anaconda/envs/data-science/lib/python3.10/site-packages/dsgtools/azure/adls.py:239: DtypeWarning: Columns (17,18,33,48,49,65,67,499,500,503,510,559,560,561,562,563,564,604,605,606) have mixed types. Specify dtype option on import or set low_memory=False.
  data = reader(file, **kwargs)
/anaconda/envs/data-science/lib/python3.10/site-packages/dsgtools/azure/adls.py:239: DtypeWarning: Columns (111,2129) have mixed types. Specify dtype option on import or set low_memory=False.
  data = reader(file, **kwargs)
/anaconda/envs/data-science/lib/python3.10/site-packages/dsgtools/azure/adls.py:239: DtypeWarning: Columns (111,2129) have mixed types. Specify dtype option on import or set low_memory=False.
  data = reader(file, **kwargs)


(50563, 2587)
(50562, 2131)
(50125, 2131)


In [8]:
shell = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_282_286/data/0627_business_shell_v21_results_w20230626-104524_busshell.csv", encoding='iso-8859-1')
print(shell.shape)

/anaconda/envs/data-science/lib/python3.10/site-packages/dsgtools/azure/adls.py:239: DtypeWarning: Columns (37,45,53,54,77,93,95,97,285,301,326,343,345,838,839,840,979,996,997,1005,1067,1068,1071,1072,1075,1076,1470,3965) have mixed types. Specify dtype option on import or set low_memory=False.
  data = reader(file, **kwargs)


(50563, 3966)


In [9]:
fmt_1 = make_format(cuts = [-np.inf, 0, np.inf])
fmt_2 = make_format(cuts = [-np.inf, 0, 43, np.inf])
freq(shell['verification.inputidmatchseleid'].astype(float), shell.seleidweight.astype(float), format = [fmt_1, fmt_2], cross=False, observed = True)

Count       Pct  Cuml Count  \
verification.inputidmatchseleid seleidweight                                  
<= 0                            <= 0           8437.0  0.166861      8437.0   
1+                              1-43            255.0  0.005043      8692.0   
                                44+           41866.0  0.827997     50558.0   
Missing                         <= 0              5.0  0.000099     50563.0   

                                              Cuml Pct  
verification.inputidmatchseleid seleidweight            
<= 0                            <= 0          0.166861  
1+                              1-43          0.171904  
                                44+           0.999901  
Missing                         <= 0          1.000000

In [10]:
shell.columns

Index(['ultidweight', 'ultidscore', 'orgidweight', 'orgidscore',
       'seleidweight', 'seleidscore', 'proxidweight', 'proxidscore',
       'powidweight', 'powidscore',
       ...
       'jpmc.jpmctotcredremmax', 'jpmc.jpmctotalcurcredlmt',
       'jpmc.jpmccurrentutilrevnocf', 'jpmc.jpmccurrentutilcardnocf',
       'jpmc.jpmcutilfin12mnocf', 'jpmc.jpmcutilrev3mnocf',
       'jpmc.jpmcsumvelocityavg11mrev', 'jpmc.jpmcsumvelocityavg48mrev',
       'jpmc.filteredsbfeaccountcount', 'errorcode'],
      dtype='object', length=3966)

In [11]:
shell["change_flag"] = np.where((shell['verification.inputidmatchseleid'] > 0) & (shell.seleidweight < 44), 1, 0)
freq(shell.change_flag)

,Count,Pct,Cuml Count,Cuml Pct
change_flag,,,,
0,50308.0,0.994957,50308.0,0.994957
1,255.0,0.005043,50563.0,1.000000


In [12]:
shell = shell[["input_echo.acctno", "verification.inputidmatchseleid", "seleidweight", "change_flag"]]
baseline = baseline.merge(shell, left_on = "accountnumber", right_on = 'input_echo.acctno')
test = test.merge(shell, left_on = "accountnumber", right_on = 'input_echo.acctno')

print(baseline.shape)
print(test.shape)

(50562, 2135)
(50125, 2135)


In [13]:
baseline["seleid_same"] = np.where(baseline.seleid == baseline["verification.inputidmatchseleid"], 1, 0)
freq(baseline["seleid_same"], baseline.change_flag, cross = False, observed = True)

Count       Pct  Cuml Count  Cuml Pct
seleid_same change_flag                                         
1           0            50289.0  0.994601     50289.0  0.994601
            1                3.0  0.000059     50292.0  0.994660
0           0               18.0  0.000356     50310.0  0.995016
            1              252.0  0.004984     50562.0  1.000000

In [14]:
test["seleid_same"] = np.where(test.seleid == test["verification.inputidmatchseleid"], 1, 0)
freq(test["seleid_same"], test.change_flag, cross = False, observed = True)

Count       Pct  Cuml Count  Cuml Pct
seleid_same change_flag                                         
1           0            49855.0  0.994613     49855.0  0.994613
            1              208.0  0.004150     50063.0  0.998763
0           0               18.0  0.000359     50081.0  0.999122
            1               44.0  0.000878     50125.0  1.000000

In [22]:
samples

,accountnumber,issue
0,AAA000000101648979,issue1
1,AAA000000103785171,issue1
2,AAA000000101239929,issue1
3,AAA000000116743692,issue2
4,AAA000000102369306,issue2
...,...,...
92,AAA000000117346363,good_test
93,AAA000000117370678,good_test
94,AAA000000117242372,good_test
95,172863881R500496,good_test


In [21]:
issue1 = pd.DataFrame(baseline.loc[(baseline["seleid_same"] ==1) & (baseline.change_flag ==1), "accountnumber"])
issue1["issue"] = "issue1"
issue2 = pd.DataFrame(test.loc[(test["seleid_same"] ==0) & (test.change_flag ==1), "accountnumber"])
issue2["issue"] = "issue2"
good21 = pd.DataFrame(test.loc[(test["seleid_same"] ==1) & (test.change_flag ==1), "accountnumber"].sample(30, random_state = 1))
good22 = pd.DataFrame(test.loc[(test["seleid_same"] ==1) & (test.change_flag ==0), "accountnumber"].sample(10, random_state = 1))
good23 = pd.DataFrame(test.loc[(test["seleid_same"] ==0) & (test.change_flag ==0), "accountnumber"].sample(10, random_state = 1))
good21["issue"] = "good_test"
good22["issue"] = "good_test"
good23["issue"] = "good_test"

samples = pd.concat([issue1, issue2, good21, good22, good23], ignore_index=True)
samples.to_csv("./_temp/samples_w_flag_282.csv", index = False)

In [30]:
baseline.loc[(baseline["seleid_same"] == 1) & (baseline.change_flag == 1), ["accountnumber", 'seleid', "verification.inputidmatchseleid", "seleidweight", "model2score"]]

,accountnumber,seleid,verification.inputidmatchseleid,seleidweight,model2score
10104,AAA000000101648979,139054446365,1.390544e+11,41,759
19590,AAA000000103785171,139220304577,1.392203e+11,43,222
42463,AAA000000101239929,136571832076,1.365718e+11,42,222


In [31]:
test.loc[(test["seleid_same"] == 0) & (test.change_flag == 1), ["accountnumber", 'seleid', "verification.inputidmatchseleid", "seleidweight"]]

,accountnumber,seleid,verification.inputidmatchseleid,seleidweight
1,AAA000000116743692,0,1.387987e+11,43
3,AAA000000102369306,0,1.351749e+11,42
1008,AAA000000101967240,0,1.339872e+11,40
1016,AAA000000114130175,0,1.363396e+11,43
1994,AAA000000117238228,0,1.363415e+11,43
3011,AAA000000101186587,0,1.387088e+11,41
3013,AAA000000102900946,0,1.388021e+11,40
9000,AAA000000113831237,0,1.388013e+11,42
10000,AAA000000118782387,0,7.358946e+07,42
10003,AAA000000103858147,0,1.373228e+11,40


Bad pipe message: %s [b'\x15\xc3YM\x03\xe4\xd1\x82\x02g\x86~\xefw"\xbe\x85x \r&\x00\xacv6\xdb8\x11E4oD9{Co\xd6\xd4\xdcPg,\xc9\xe4\xa4\xceM\x84\xb2\xd2\xde\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e']
Bad pipe message: %s [b'\xb5Q\xed/\xbe\x86G\xea\xcb\xdf0)\xac\x02\xf0L]\xef ~\xda\xf7s\xbfoO\xc0\x951\xfb\xe5\x90\xd0\xf5\xdc\x99\xaf\x8f\xa9\xee\xf3\xe9{=\xfd\x97`t\xc6J%\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127']
Bad pipe message: %s [b'.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00']
Bad pipe message: %s [b'\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01']
Bad pipe 

In [ ]:
import datacompy

In [ ]:
# base_no_impact = baseline[(baseline["seleid_same"] == 1) & (baseline["change_flag"] == 0)].copy()
# test_no_impact = test[(test["seleid_same"] == 1) & (test["change_flag"] == 0)].copy()

# scout_compare = datacompy.Compare(
#             base_no_impact,
#             test_no_impact,
#             join_columns='accountnumber', #You can also specify a list of columns
#             abs_tol=0.0001,
#             rel_tol=0,
#             df1_name='base',
#             df2_name='test')

wb = TableWriter(filename = "./_temp/scout_no_impact.xlsx", options={'nan_inf_to_errors': True})
wb.write_table(
    pd.DataFrame(scout_compare.column_stats).fillna("n/a"),
    sheetname="scout_no_impact",
)
wb.close() 

In [ ]:
# print(scout_compare.report())
from dsgtools.reporting import col_summary

wb = TableWriter(filename = "./_temp/change_baseline.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
wb.write_table(
    col_summary(baseline[(baseline["change_flag"] == 1)].copy(), exceptions = [-1, -97, -98, -99]).T.fillna(""),
    sheetname="change_baseline",
)
wb.close() 

In [ ]:
test[(test["change_flag"] == 1)].columns

Index(['accountnumber', 'historydateyyyymm', 'bus_company_name', 'powid',
       'proxid', 'seleid', 'orgid', 'ultid', 'lnlexidsele',
       'inputcheckbusname',
       ...
       'model10rc4', 'model10rc5', 'model10rc6', 'errorcode', 'time_ms',
       'account', 'id_seleid', 'id_weight', 'change_flag', 'seleid_same'],
      dtype='object', length=2309)

### Business Shell 22

In [ ]:
shell = pd.read_adls(path + "data/business_shell_v22_results_w20230614-114025_sas_layout_busshell.csv", encoding='iso-8859-1')
test = pd.read_adls(path + "data/sba_v1_sbfe_busshell_w20230614-153259", encoding='iso-8859-1', reader = pd.read_csv)
baseline = pd.read_adls(path + "data/sba_v1_sbfe_busshell_w20230614-134050", encoding='iso-8859-1', reader = pd.read_csv)

print(shell.shape)
print(baseline.shape)
print(test.shape)

/anaconda/envs/data-science/lib/python3.10/site-packages/dsgtools/azure/adls.py:235: DtypeWarning: Columns (19,20,36,51,52,69,71,603,652,653,654,655,656,657,697,698,699) have mixed types. Specify dtype option on import or set low_memory=False.
  data = reader(file, **kwargs)
/anaconda/envs/data-science/lib/python3.10/site-packages/dsgtools/azure/adls.py:235: DtypeWarning: Columns (111,2129) have mixed types. Specify dtype option on import or set low_memory=False.
  data = reader(file, **kwargs)
/anaconda/envs/data-science/lib/python3.10/site-packages/dsgtools/azure/adls.py:235: DtypeWarning: Columns (111,2129) have mixed types. Specify dtype option on import or set low_memory=False.
  data = reader(file, **kwargs)


(50555, 2700)
(50560, 2131)
(50555, 2131)


In [ ]:
fmt_1 = make_format(cuts = [-np.inf, 0, np.inf])
fmt_2 = make_format(cuts = [-np.inf, 0, 43, np.inf])
freq(shell.id_seleid.astype(float), shell.id_weight.astype(float), format = [fmt_1, fmt_2], cross=False, observed = True)

Count       Pct  Cuml Count  Cuml Pct
id_seleid id_weight                                         
<= 0      <= 0        8436.0  0.166868      8436.0  0.166868
1+        1-43         255.0  0.005044      8691.0  0.171912
          44+        41859.0  0.827989     50550.0  0.999901
Missing   Missing        5.0  0.000099     50555.0  1.000000

In [ ]:
shell["change_flag"] = np.where((shell.id_seleid > 0) & (shell.id_weight < 44), 1, 0)
freq(shell.change_flag)

,Count,Pct,Cuml Count,Cuml Pct
change_flag,,,,
0,50300.0,0.994956,50300.0,0.994956
1,255.0,0.005044,50555.0,1.000000


In [15]:
shell = shell[["account", "id_seleid", "id_weight", "change_flag", "id_truebiz"]]
baseline = baseline.merge(shell, left_on = "accountnumber", right_on = 'account')
test = test.merge(shell, left_on = "accountnumber", right_on = 'account')

In [16]:
print(baseline.shape)
print(test.shape)

(50552, 2136)
(50547, 2136)


In [17]:
baseline["seleid_same"] = np.where(baseline.seleid == baseline.id_seleid, 1, 0)
freq(baseline["seleid_same"], baseline.change_flag, cross = False, observed = True)

Count       Pct  Cuml Count  Cuml Pct
seleid_same change_flag                                         
1           0            50279.0  0.994600     50279.0  0.994600
            1                3.0  0.000059     50282.0  0.994659
0           0               18.0  0.000356     50300.0  0.995015
            1              252.0  0.004985     50552.0  1.000000

In [18]:
pd.set_option('display.max_columns', None)
baseline[(baseline.seleid_same == 1) & (baseline.change_flag == 1)]

accountnumber  historydateyyyymm  \
10103  AAA000000101648979             202304   
19186  AAA000000103785171             202304   
42453  AAA000000101239929             202304   

                                        bus_company_name         powid  \
10103                RICHIE AUTO TRANSPORT LOGISTICS LLC  139054446365   
19186  RÃÂÃÂÃÂÃÂVÃÂÃÂÃÂÃÂLÃÂÃÂÃÂ...  139220304577   
42453                 REVELS CHARLOTTE, LLC : REVEL ROOM  136571832076   

             proxid        seleid         orgid         ultid  \
10103  139054446365  139054446365  139054446365  139054446365   
19186  139220304577  139220304577  139220304577  139220304577   
42453  136571832076  136571832076  136571832076  136571832076   

       inputcheckbusname  inputcheckbusaltname  inputcheckbusaddr  \
10103                1.0                   0.0                1.0   
19186                1.0                   0.0                1.0   
42453                1.0                   0.0                1.0   

       inputcheckbuscity  inputcheckbusstate  inputcheckbuszip  \
10103                1.0                 1.0               1.0   
19186                1.0                 1.0               1.0   
42453                1.0                 1.0               1.0   

       inputcheckbusfein  inputcheckbusphone  inputcheckbussic  \
10103                0.0                 0.0               0.0   
19186                0.0                 0.0               0.0   
42453                0.0                 0.0               0.0   

       inputcheckbusnaics  inputcheckbusstructure  inputcheckbusage  \
10103                 0.0                     0.0               0.0   
19186                 0.0                     0.0               0.0   
42453                 0.0                     0.0               0.0   

       inputcheckbusstartdate  inputcheckbusannualrevenue  inputcheckbusfax  \
10103                     0.0                         0.0               0.0   
19186                     0.0                         0.0               0.0   
42453                     0.0                         0.0               0.0   

       inputcheckauthrepfirstname  inputcheckauthreplastname  \
10103                         0.0                        0.0   
19186                         0.0                        0.0   
42453                         0.0                        0.0   

       inputcheckauthrepmiddlename  inputcheckauthrepaddr  \
10103                          0.0                    0.0   
19186                          0.0                    0.0   
42453                          0.0                    0.0   

       inputcheckauthrepcity  inputcheckauthrepstate  inputcheckauthrepzip  \
10103                    0.0                     0.0                   0.0   
19186                    0.0                     0.0                   0.0   
42453                    0.0                     0.0                   0.0   

       inputcheckauthrepssn  inputcheckauthrepphone  inputcheckauthrepdob  \
10103                   0.0                     0.0                   0.0   
19186                   0.0                     0.0                   0.0   
42453                   0.0                     0.0                   0.0   

       inputcheckauthrepage  inputcheckauthreptitle  inputcheckauthrepdl  \
10103                   0.0                     0.0                  0.0   
19186                   0.0                     0.0                  0.0   
42453                   0.0                     0.0                  0.0   

       inputcheckauthrepdlstate  inputcheckauthrep2firstname  \
10103                       0.0                          0.0   
19186                       0.0                          0.0   
42453                       0.0                          0.0   

       inputcheckauthrep2lastname  inputcheckauthrep2middlename  \
10103                         0.0                           0.0   
19186                         0.

In [19]:
test["seleid_same"] = np.where(test.seleid == test.id_seleid, 1, 0)
freq(test["seleid_same"], test.change_flag, cross = False, observed = True)

Count       Pct  Cuml Count  Cuml Pct
seleid_same change_flag                                         
1           0            50274.0  0.994599     50274.0  0.994599
            1              210.0  0.004155     50484.0  0.998754
0           0               18.0  0.000356     50502.0  0.999110
            1               45.0  0.000890     50547.0  1.000000

In [21]:
freq("seleid", df = test[(test["change_flag"] ==1) & (test["seleid_same"] ==0)])

,Count,Pct,Cuml Count,Cuml Pct
seleid,,,,
0,45.0,1.0,45.0,1.0


In [23]:
freq("id_truebiz", df = test[(test["change_flag"] ==1) & (test["seleid_same"] ==0)])

,Count,Pct,Cuml Count,Cuml Pct
id_truebiz,,,,
0.0,30.0,0.666667,30.0,0.666667
1.0,15.0,0.333333,45.0,1.000000


In [22]:
test[(test["change_flag"] ==1) & (test["seleid_same"] ==0)].head(3)

accountnumber  historydateyyyymm              bus_company_name  \
1     AAA000000116743692             202304         EI ELITE DISPATCH LLC   
3     AAA000000102369306             202304  HAIR BY CELINA MARIE TREVINO   
1007  AAA000000101967240             202304          THE BRIDGE EL PUENTE   

      powid  proxid  seleid  orgid  ultid  inputcheckbusname  \
1         0       0       0      0      0                1.0   
3         0       0       0      0      0                1.0   
1007      0       0       0      0      0                1.0   

      inputcheckbusaltname  inputcheckbusaddr  inputcheckbuscity  \
1                      0.0                1.0                1.0   
3                      0.0                1.0                1.0   
1007                   0.0                1.0                1.0   

      inputcheckbusstate  inputcheckbuszip  inputcheckbusfein  \
1                    1.0               1.0                0.0   
3                    1.0               1.0                0.0   
1007                 1.0               1.0                0.0   

      inputcheckbusphone  inputcheckbussic  inputcheckbusnaics  \
1                    0.0               0.0                 0.0   
3                    0.0               0.0                 0.0   
1007                 0.0               0.0                 0.0   

      inputcheckbusstructure  inputcheckbusage  inputcheckbusstartdate  \
1                        0.0               0.0                     0.0   
3                        0.0               0.0                     0.0   
1007                     0.0               0.0                     0.0   

      inputcheckbusannualrevenue  inputcheckbusfax  \
1                            0.0               0.0   
3                            0.0               0.0   
1007                         0.0               0.0   

      inputcheckauthrepfirstname  inputcheckauthreplastname  \
1                            0.0                        0.0   
3                            0.0                        0.0   
1007                         0.0                        0.0   

      inputcheckauthrepmiddlename  inputcheckauthrepaddr  \
1                             0.0                    0.0   
3                             0.0                    0.0   
1007                          0.0                    0.0   

      inputcheckauthrepcity  inputcheckauthrepstate  inputcheckauthrepzip  \
1                       0.0                     0.0                   0.0   
3                       0.0                     0.0                   0.0   
1007                    0.0                     0.0                   0.0   

      inputcheckauthrepssn  inputcheckauthrepphone  inputcheckauthrepdob  \
1                      0.0                     0.0                   0.0   
3                      0.0                     0.0                   0.0   
1007                   0.0                     0.0                   0.0   

      inputcheckauthrepage  inputcheckauthreptitle  inputcheckauthrepdl  \
1                      0.0                     0.0                  0.0   
3                      0.0                     0.0                  0.0   
1007                   0.0                     0.0                  0.0   

      inputcheckauthrepdlstate  inputcheckauthrep2firstname  \
1                          0.0                          0.0   
3                          0.0                          0.0   
1007                       0.0                          0.0   

      inputcheckauthrep2lastname  inputcheckauthrep2middlename  \
1                            0.0                           0.0   
3                            0.0                           0.0   
1007                         0.0                           0.0   

      inputcheckauthrep2addr  inputcheckauthrep2city  inputcheckauthrep2state  \
1                        0.0                     0.0                      0.0   
3                        0.0               

In [28]:
# shell = pd.read_adls(path + "data/business_shell_v22_results_w20230614-114025_sas_layout_busshell.csv", encoding='iso-8859-1')
shell.loc[shell["account"] == "AAA000000101967240", "ver_src_id_list"]

46219    IN
Name: ver_src_list, dtype: object

In [29]:
test.loc[(test["change_flag"] ==1) & (test["seleid_same"] ==0), "accountnumber"].to_csv("debug_282_0621.csv", index = False)

Bad pipe message: %s [b'(V-m\xde@\xd5\xde\xbe!\xb4\x9a\x15\x9b@w\xa5)\x00\x00\xa2\xc0\x14\xc0\n\x009\x008\x007\x006\x00\x88\x00\x87\x00\x86\x00\x85\xc0\x19\x00:\x00\x89\xc0\x0f\xc0\x05\x005\x00\x84\xc0\x13\xc0\t\x003\x002\x001\x000\x00\x9a\x00\x99\x00\x98\x00\x97\x00E\x00D\x00C\x00B\xc0\x18\x004\x00\x9b\x00F\xc0\x0e\xc0\x04\x00/\x00\x96\x00A\x00\x07\xc0\x11\xc0\x07\xc0\x16\x00\x18\xc0\x0c\xc0\x02\x00\x05\x00\x04\xc0\x12\xc0\x08\x00\x16\x00\x13\x00\x10\x00\r\xc0\x17\x00\x1b\xc0\r\xc0\x03\x00\n\x00\x15\x00\x12\x00\x0f\x00\x0c\x00\x1a\x00\t\x00\x14\x00\x11']
Bad pipe message: %s [b"O\xc2z'\x1c\x90\xeb\xa7\xfc\xed\xb6\x1d\xf2\xb2\xe4\x80\x9f\xf2\x00\x00\xa2\xc0\x14\xc0\n\x009\x008\x007\x006\x00\x88\x00\x87\x00\x86\x00\x85\xc0\x19\x00:\x00\x89\xc0\x0f\xc0\x05\x005\x00\x84\xc0\x13\xc0\t\x003\x002\x001\x000\x00\x9a\x00\x99\x00\x98\x00\x97\x00E\x00D\x00C\x00B\xc0\x18\x004\x00\x9b\x00F\xc0\x0e\xc0\x04\x00/\x00\x96\x00A\x00\x07\xc0\x11\xc0\x07\xc0\x16\x00\x18\xc0\x0c\xc0\x02\x00\x05\x00\x04\xc0\

In [ ]:
import datacompy

In [ ]:
# base_no_impact = baseline[(baseline["seleid_same"] == 1) & (baseline["change_flag"] == 0)].copy()
# test_no_impact = test[(test["seleid_same"] == 1) & (test["change_flag"] == 0)].copy()

# scout_compare = datacompy.Compare(
#             base_no_impact,
#             test_no_impact,
#             join_columns='accountnumber', #You can also specify a list of columns
#             abs_tol=0.0001,
#             rel_tol=0,
#             df1_name='base',
#             df2_name='test')

wb = TableWriter(filename = "./_temp/scout_no_impact.xlsx", options={'nan_inf_to_errors': True})
wb.write_table(
    pd.DataFrame(scout_compare.column_stats).fillna("n/a"),
    sheetname="scout_no_impact",
)
wb.close() 

In [ ]:
# print(scout_compare.report())
from dsgtools.reporting import col_summary

wb = TableWriter(filename = "./_temp/change_baseline.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
wb.write_table(
    col_summary(baseline[(baseline["change_flag"] == 1)].copy(), exceptions = [-1, -97, -98, -99]).T.fillna(""),
    sheetname="change_baseline",
)
wb.close() 

In [ ]:
test[(test["change_flag"] == 1)].columns

Index(['accountnumber', 'historydateyyyymm', 'bus_company_name', 'powid',
       'proxid', 'seleid', 'orgid', 'ultid', 'lnlexidsele',
       'inputcheckbusname',
       ...
       'model10rc4', 'model10rc5', 'model10rc6', 'errorcode', 'time_ms',
       'account', 'id_seleid', 'id_weight', 'change_flag', 'seleid_same'],
      dtype='object', length=2309)